In [87]:
# %reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [88]:
# REFERENCE 

# Bokeh Plotting: Enable tooltips for only some glyphs
# https://stackoverflow.com/questions/29435200/bokeh-plotting-enable-tooltips-for-only-some-glyphs

# Bokeh Documentation: Configuring Plot Tools
# https://docs.bokeh.org/en/latest/docs/user_guide/tools.html

# Bokeh Documentation: Styling Visual Attributes
# https://docs.bokeh.org/en/latest/docs/user_guide/styling.html

In [89]:
import pandas as pd 

from bokeh.io import output_notebook, show, curdoc
from bokeh.plotting import figure
from bokeh.layouts import column, gridplot
from bokeh.models import ColumnDataSource, TapTool, OpenURL, CustomJSHover, Circle, Line, Text
from bokeh.models.tickers import DatetimeTicker
from bokeh.models.tools import HoverTool, PanTool, BoxZoomTool, WheelZoomTool, SaveTool, ResetTool, TapTool

from sklearn.preprocessing import MinMaxScaler

In [90]:
fileName = '200927_ParsedVideoData_v4.csv'
data = pd.read_csv(fileName)

data.publishedAt = pd.to_datetime(data.publishedAt)
data.duration = pd.to_timedelta(data.duration)

In [91]:
dataColor1 = '#003f5c'
dataColor2 = '#7a5195'
dataColor3 = '#ef5675'
dataColor4 = '#ffa600'

In [92]:
# normalize data for plotting
data_columns = ['viewCount', 'likeCount', 'dislikeCount', 'commentCount']
source_columns = [x +'Norm' for x in data_columns]
data_normalized = pd.DataFrame(MinMaxScaler().fit_transform(data[data_columns]), columns=source_columns)
data = pd.concat([data, data_normalized], axis=1)

In [114]:
source = ColumnDataSource(data)

data_colors = [dataColor3, dataColor4, dataColor1, dataColor2]

# url to open on click
url = 'https://www.youtube.com/watch?v=@id'

# configure hover tooltips
hover_data = [('Guest', '@guestName'),
             ('Number', '@videoNumber{0,0}'),
             ('Aired On', '@publishedAt{%F}'),
             ('View', '@viewCount{0,0.0 a}'),
             ('Likes', '@likeCount{0,0.0 a}'),
             ('Dislikes', '@dislikeCount{0,0.0 a}'),
             ('Comments', '@commentCount{0,0.0 a}'),
            ]

figure_titles = ['Views', 'Likes', 'Dislikes', 'Comments']

def create_figures(source, source_columns, fig_len=1200, fig_height=150, data_colors=data_colors, hover_data=hover_data, alpha=0.5):
    figure_collector = []
    num_figures = len(source_columns)
    num = 0
    

    while num < num_figures:
        # create figure
        fig = figure(plot_width=fig_len, plot_height=fig_height, x_axis_type='datetime', name='data_series')
        
        # add tools
        tools = [PanTool(), BoxZoomTool(), WheelZoomTool(), SaveTool(), ResetTool()]
        fig.add_tools(*tools)
        
        # create the scatter plot 
        scatter = Circle(x='publishedAt', y=source_columns[num], line_color=data_colors[num], fill_color=data_colors[num], \
                         size=6, fill_alpha=alpha, line_alpha=alpha
                        )
        scatter_render = fig.add_glyph(source_or_glyph=source, glyph=scatter)
        # hover only over scatter plot
        hover = HoverTool(renderers=[scatter_render], tooltips=hover_data, formatters={'@publishedAt':'datetime'})
        fig.add_tools(hover)
        # open video url on tap scatter points
        taptool = TapTool(renderers=[scatter_render])
        taptool.callback = OpenURL(url=url)
        fig.add_tools(taptool)
    
        # create line plot without hover or top 
        line = Line(x='publishedAt', y=source_columns[num], line_color=data_colors[num], line_width=1, line_alpha=alpha)
        line_render = fig.add_glyph(source_or_glyph=source, glyph=line)
        
        # add series title
        title = Text(x=data['publishedAt'].min(), y=0.2, \
                     text=[figure_titles[num]], text_color=data_colors[num], text_font_size='35px')
        fig.add_glyph(title)
                
        # decrease clutter
        fig.outline_line_color = None
        fig.xgrid.grid_line_color = None
        fig.ygrid.grid_line_color = None
        fig.yaxis.visible = False
        fig.xaxis.visible = False   
        
        
        # format x-axis ticks 
        fig.xaxis.major_tick_line_color = "grey"
        fig.xaxis.major_label_text_font_size = "12pt"
        fig.xaxis.major_label_text_color = "grey"

        
        # collect figures in a list 
        figure_collector.append(fig)
        num+=1
    return figure_collector

figures = create_figures(source, source_columns)

# show x-axis on the bottom figure
figures[-1].xaxis.visible = True
figures[-1].xaxis.ticker = DatetimeTicker(desired_num_ticks=10)
figures[-1].xaxis.axis_line_color = 'grey'



# # share last figures x-axis with all figures to allow linked zoom and pan
for fig in figures[:-1]: 
    fig.x_range = figures[-1].x_range
    fig.y_range = figures[-1].y_range


layout = gridplot(figures, ncols=1)
curdoc().add_root(layout)

# uncomment if viewing on Jupyter Notebook
output_notebook()
show(layout)

Loading BokehJS ...